In [36]:
pip install qiskit==1.1.2 qiskit-optimization qiskit_aer qiskit_utils


In [37]:
pip install qiskit_algorithms

In [38]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.optimizers import COBYLA
from qiskit.compiler import transpile, assemble
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [39]:
# --- Inputs ---
q = 0.05      #risk tollerance
P = 100       #Penalty weight
mu = [0.08745401 , 0.14507143 , 0.12319939 ,0.10986585, 0.06560186, 0.06559945,
 0.05580836, 0.13661761, 0.1101115 , 0.12080726]   #profits from each properties
sigma = [[0.02058449 , 0.96990985 , 0.83244264 , 0.21233911 ,0.18182497 ,0.18340451,
            0.30424224, 0.52475643, 0.43194502, 0.29122914],
        [0.61185289, 0.13949386, 0.29214465, 0.36636184 , 0.45606998, 0.78517596,
            0.19967378, 0.51423444, 0.59241457, 0.04645041],
        [0.60754485, 0.17052412, 0.06505159, 0.94888554, 0.96563203, 0.80839735,
            0.30461377, 0.09767211, 0.68423303, 0.44015249],
        [0.12203823, 0.49517691, 0.03438852, 0.9093204,  0.25877998 ,0.66252228,
            0.31171108, 0.52006802, 0.54671028, 0.18485446],
        [0.96958463, 0.77513282, 0.93949894, 0.89482735, 0.59789998, 0.92187424,
            0.0884925,  0.19598286, 0.04522729, 0.32533033],
        [0.38867729, 0.27134903, 0.82873751, 0.35675333, 0.28093451,0.54269608,
            0.14092422, 0.80219698, 0.07455064, 0.98688694],
        [0.77224477, 0.19871568, 0.00552212, 0.81546143, 0.70685734, 0.72900717,
            0.77127035, 0.07404465, 0.35846573 ,0.11586906],
        [0.86310343, 0.62329813, 0.33089802 ,0.06355835 ,0.31098232, 0.32518332,
            0.72960618, 0.63755747, 0.88721274, 0.47221493],
        [0.11959425 ,0.71324479, 0.76078505, 0.5612772,  0.77096718, 0.4937956,
            0.52273283, 0.42754102, 0.02541913, 0.10789143],
        [0.03142919 ,0.63641041, 0.31435598 ,0.50857069, 0.90756647, 0.24929223,
            0.41038292, 0.75555114, 0.22879817, 0.07697991]]    #coorelation matrix

k = 3 # Desired number of properties
n=len(mu)  #Number of available properties

The given QUBO is a  derivation of Markowitz Optimization Algorithm the orginal algorithm aims to minimize the cost involved in insuring a property. Which is obtained by subtracting the resturen from a property subtracted from the risk associated with them. Note we do have a budject constraint where a company can insure only certain number of properties. We form a QUBO exquation by introducting the penalty term into the orginal exuation. min⁡〖q∑_i▒∑_j▒〖x_i x_j σ_ij 〗〗- ∑_i▒〖x_i μ_i 〗
s.t.∑_i▒x_i =B

	q>0 is the risk appetite of the decision maker. 5% is a reasonable value for this case.
	x_i is the decision variable indicating whether property i is included in the portfolio.
	σ_ij represents the correlation between properties i and j.
	μ_i is the expected return on the property i, i.e., the premium, which we assume to be fixed (and given) in this case.
	B is the budget, i.e., how many properties a decision maker wants in its portfolio. 10% of the possible properties is a reasonable value for this case.
min⁡〖q∑_i▒∑_j▒〖x_i x_j σ_ij 〗〗- ∑_i▒〖x_i μ_i 〗+ k(∑_i▒x_i -B)^2.with K being the whole number (bigger as possible). After expanding the Penalty term the equation for pernalty becomes kx^2 + 2xixjK + K^2. We can eliminate the cooficient term as it does not affect the QUBO diagonal. The kx^2 term affects the diagonal terms(where i==j) since there is no connection. the k(1 - 2*xi*xj) will be affecting the non-diagonal terms.

In [40]:
#Uses simulated anneling which as a quantum inspired classical algorithm

# Step 1: Build QUBO matrix Q
Q = {}
for i in range(n):
    Q[(i, i)] = q * sigma[i][i] - mu[i] + P * (1 - 2 * k)  #set up the diagonal cost (include penalty)
    for j in range(i + 1, n):
        Q[(i, j)] = q * sigma[i][j] + 2 * P     #set up the diagonal cost (include penalty)

# Step 2: Energy calculation
def energy(x, Q):
    e = 0
    for i in range(n):
        e += Q.get((i, i), 0) * x[i]
        for j in range(i + 1, n):
            e += Q.get((i, j), 0) * x[i] * x[j]
    return e

# Step 3: Simulated Annealing
#Do repeat the anneleing proccess until the specifed steps are untild the system reacheds its colleset point
#temp_start-> the starting temperature of the system end->lowest temperate a system can reach alpha->the rate at which the system looses its temp.
def simulated_annealing(Q, n, steps=1000, temp_start=100.0, temp_end=1e-2, alpha=0.98):

    # Initial solution
    current = [random.randint(0, 1) for _ in range(n)]
    current_energy = energy(current, Q)
    best = list(current)
    best_energy = current_energy
    temp = temp_start

    for step in range(steps):
        # Flip one bit
        i = random.randint(0, n-1)
        neighbor = list(current)
        neighbor[i] = 1 - neighbor[i]
        neighbor_energy = energy(neighbor, Q)

        delta = neighbor_energy - current_energy
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current = neighbor
            current_energy = neighbor_energy
            if current_energy < best_energy:
                best = current
                best_energy = current_energy

        temp *= alpha
        if temp < temp_end:
            break

    return best, best_energy

# Run the solver
best_solution, best_energy = simulated_annealing(Q, n)

# Show result
selected_props = [chr(65 + i) for i in range(n) if best_solution[i] == 1]
print("Best selection:", best_solution)
print("Energy (objective value):", best_energy)
print("Selected properties:", selected_props)


Best selection: [1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
Energy (objective value): -900.207809356
Selected properties: ['A', 'H', 'I']


In [41]:

#Uses QAOA built in algorithm from qiskit to so;ve the given QUBO

# Step 1:QUBO Encoding
qp = QuadraticProgram()
for i in range(n):
    qp.binary_var(name=f'x{i}')

# Objective function: combine risk, return, and constraint penalty
linear = {}
quadratic = {}

# Setp 2:Risk-return terms
for i in range(n):
    linear[f'x{i}'] = q * sigma[i][i] - mu[i] + P * (1 - 2 * k)
    for j in range(i + 1, n):
        key = (f'x{i}', f'x{j}')
        quadratic[key] = q * sigma[i][j] + 2 * P

qp.minimize(linear=linear, quadratic=quadratic)

# Step 3: Solve using QAOA (Quantum Annealing behavior)
algorithm_globals.random_seed = 42
sampler = AerSampler()
qaoa = QAOA(sampler=sampler,optimizer=COBYLA(), reps=1) # Increase the number of reps to 100
optimizer = MinimumEigenOptimizer(qaoa)

result = optimizer.solve(qp)

# Step 4: Show Result
solution = result.x
selected_props = [chr(65 + i) for i in range(n) if solution[i] == 1]

print("Best selection (quantum):", solution)
print("Energy (objective value):", result.fval)
print("Selected properties:", selected_props)


Best selection (quantum): [0. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Energy (objective value): -900.3360644344999
Selected properties: ['B', 'C', 'J']


In [42]:
#Solve the QUBO using quantum circuit and gates
#Implementation of QAOA algorithm with n qbit system

Q = {}
for i in range(n):
    Q[(i, i)] = q * sigma[i][i] - mu[i] + P * (1 - 2 * k)
    for j in range(i + 1, n):
        Q[(i, j)] = q * sigma[i][j] + 2 * P

# Quantum circuit with custom annealing steps
p = 3  # number of layers more layers more precise output but more complex
gamma = 0.8
beta = 0.7

qc = QuantumCircuit(n, n)

# Step 1: Initialize in superposition
qc.h(range(n)) # Hadamard Gate

# Step 2: p-layer QAOA-style evolution
for _ in range(p):
    # Cost unitary (phase encoding)
    for (i, j), qval in Q.items():
        if i == j:    #diagonal element z-phase rotation, peeks the how costly it may get to select this bit (dont select them)
            qc.rz(2 * gamma * qval, i)
        else:   #Non-diagonal, controlled phase rotation to say about the penalty or rewards associated with a bit
            qc.cx(i, j)
            qc.rz(2 * gamma * qval, j)
            qc.cx(i, j)

    # Mixing unitary  flips the table to escape local minima and allow exploring further
    for i in range(n):
        qc.rx(2 * beta, i)

# Step 3: Measurement
qc.measure(range(n), range(n))

# Simulate
# --- Run Simulation ---
sim = Aer.get_backend('qasm_simulator')
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=2048).result()
counts = result.get_counts()

#Gate based Quantum circuits are prone to errors/noises which might not talk the penalty take into account and selects the leaset energy(greedy approch)
#We need to explicity tell the circut to pick the least energy that obeys our penalty term.
# Only consider bitstrings with Hamming weight = k
filtered_counts = {k: v for k, v in counts.items() if k.count('1') == 3}

if filtered_counts:
    best = max(filtered_counts, key=filtered_counts.get)
    best_selection = [int(b) for b in best]
    selected_props = [chr(65 + i) for i in range(n) if best_selection[i] == 1]
    print("Filtered Best Selection:", best_selection)
    print("Selected Properties:", selected_props)
else:
    print("No valid properties were found.")


Filtered Best Selection: [0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
Selected Properties: ['C', 'D', 'H']


In [43]:
#Implementationof VQA algorithm using quantum circits and gates

# Construct QUBO matrix
Q = {}
for i in range(n):
    Q[(i, i)] = q * sigma[i][i] - mu[i] + P * (1 - 2 * k)
    for j in range(i + 1, n):
        Q[(i, j)] = q * sigma[i][j] + 2 * P


In [44]:

# Construct the ansatz
# A quantum Circit that get phase shifted around y
def create_ansatz(n, params):
    circuit = QuantumCircuit(n)
    for i in range(n):
        circuit.ry(params[i], i)  # Single qubit rotation (RY gate)

    for i in range(n - 1):
        circuit.cz(i, i + 1)  # Entangling gate (ControlledZ gate)

    return circuit


In [45]:
#Calculate the energy of each bitstring('101') coming out of the QS,
#This steps kind of evaluates the penalty and rewards associated with the element and returns the average energy
def estimate_energy(counts, Q):
    energy = 0
    shots = sum(counts.values())
    for bitstring, count in counts.items():
        x = [int(bit) for bit in reversed(bitstring)]  # reversed for qiskit order
        e = 0
        for i in range(len(x)):
            e += Q.get((i, i), 0) * x[i]
            for j in range(i + 1, len(x)):
                e += Q.get((i, j), 0) * x[i] * x[j]
        energy += (count / shots) * e
    return energy


In [46]:
#Train the QS to solve the QUBO, we use gradient decient to do this.
def optimize_ansatz(n, Q, max_iter=100, lr=0.1):
    simulator = Aer.get_backend('qasm_simulator')
    params = np.random.uniform(0, 2 * np.pi, n * 2)

    def ansatz(params):
        qc = QuantumCircuit(n)
        for i in range(n):  #phase rotations in y and z
            qc.ry(params[i], i)
            qc.rz(params[n + i], i)
        qc.measure_all()
        return qc

    #Gets the enery involved
    def compute_expectation(params):
        qc = ansatz(params)
        tqc = transpile(qc, simulator)
        result = simulator.run(tqc, shots=1024).result()
        counts = result.get_counts()
        return estimate_energy(counts, Q)
    #Determine which side to move using the energy difference,
    for _ in range(max_iter):
        current_energy = compute_expectation(params)
        gradients = np.zeros_like(params)
        for i in range(len(params)):
            shift = np.zeros_like(params)
            shift[i] = np.pi / 2
            energy_plus = compute_expectation(params + shift)
            energy_minus = compute_expectation(params - shift)
            gradients[i] = (energy_plus - energy_minus) / 2
        params -= lr * gradients

    return counts,params, compute_expectation(params)


In [48]:
# Run the VQA optimization loop
counts,optimal_params, optimal_energy = optimize_ansatz(n, Q)

# Print results
print("Optimal parameters found:", optimal_params)
print("Optimal energy:", optimal_energy)

#Gate based Quantum circuits are prone to errors/noises which might not talk the penalty take into account and selects the leaset energy(greedy approch)
#We need to explicity tell the circut to pick the least energy that obeys our penalty term.
# Only consider bitstrings with Hamming weight = k
filtered_counts = {k: v for k, v in counts.items() if k.count('1') == 3}


if filtered_counts:
    best = max(filtered_counts, key=filtered_counts.get)
    best_selection = [int(b) for b in best]
    selected_props = [chr(65 + i) for i in range(n) if best_selection[i] == 1]
    print("Filtered Best Selection:", best_selection)
    print("Selected Properties:", selected_props)
else:
    print("No valid properties were found.")

Optimal parameters found: [-168.55332722  151.20639215 -131.11372016  -34.23998232  110.4557394
    7.59244078 -136.928914    -75.33273915  -30.42038931   88.04664778
    2.92229037   -7.1257881    -5.12856632    1.53527038   -3.39974579
   -8.4602286    -5.32779347  -16.23735896    4.69003358   19.82744182]
Optimal energy: -775.195928004487
Filtered Best Selection: [0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
Selected Properties: ['C', 'D', 'H']
